# ON TIME IN FULL

In [1]:
import pandas as pd

## Bước 1. Đọc dữ liệu

In [2]:
# Đọc dữ liệu PO_Record
df_PO = pd.read_excel('PO-Record.xlsm',sheet_name='PO Data')
df_PO.head(2)

,PONbr,POLineNbr,PR_NBR_REF,PR_LINE_REF,SiteID,InvtID,Trandesc,Super_Category,CuryID,UnitCost,...,Descr,PurchUnit,VendID,VendorName,Status_Rcpt,Status_Voucher,TradeTerm,Status_PO,Crtd_user_PO,Crtd_Date_PO
0,A000007101,1,FD-14090,7,02GIAVI,A20019,"Xe đua F - Đen, 66",Sản phẩm_Xe đạp,VND,120000.0,...,15 ngày,CAI,100011,Cong Ty Co Phan Thuong Mai Dich Vu Ngoc An,Full Receipt,Fully Vouchered,Delivery At Seller,Completed,HADM,2021-08-15
1,A000007101,2,FD-14090,10,02GIAVI,A20015,"Khung xe đua LL - Đỏ, 60",Phụ tùng_Xe đạp,VND,76500.0,...,15 ngày,CAI,100011,Cong Ty Co Phan Thuong Mai Dich Vu Ngoc An,Full Receipt,Fully Vouchered,Delivery At Seller,Completed,HADM,2021-08-15


## Bước 2. Lấy dữ liệu có thể tính OTIF

##### PROMDATE < TODAY

In [3]:
### LẤY PROMDATE < TODAY

from datetime import date

df_PO['PromDate'] = pd.to_datetime(df_PO['PromDate']).dt.date
#df_PO['PromDate'] = pd.to_datetime(df_PO['PromDate'], format='%Y-%m-%d')    (dtype: datetime64[ns] ****)
#print(df_PO['PromDate'])
    
today = date(2021, 4, 30)
#today = datetime.date(2021, 4, 30) (in case 'import datetime')
#today = date.today()

PromDate_filter = df_PO['PromDate'] < today

df_PO_OTIF = df_PO[PromDate_filter][['PONbr', 'POLineNbr', 'QtyOrd', 'QtyRcvd', 'PromDate']]
#df_PO_OTIF = df_PO.loc[PromDate_filter][['PONbr', 'POLineNbr', 'QtyOrd', 'QtyRcvd', 'PromDate']]
df_PO_OTIF

,PONbr,POLineNbr,QtyOrd,QtyRcvd,PromDate
6,A000005177,1,2700.0,2700.0,2021-04-20
7,A000005022,1,1000.0,1000.0,2021-04-06
8,A000004594,1,500.0,500.0,2021-02-08
9,A000004491,1,10000.0,10003.0,2021-01-15
10,A000004429,1,2000.0,1988.3,2021-01-30
...,...,...,...,...,...
4202,A000003984,1,1.0,1.0,2021-01-21
4203,A000003984,2,2.0,2.0,2021-01-21
4204,A000003984,3,2.0,2.0,2021-01-21
4205,A000003984,4,1.0,1.0,2021-01-21


##### OTIF column is BLANK

In [4]:
### Filter OTIF Blank if 'OTIF' is existed in data. Otherwise, continue

if 'OTIF' not in df_PO_OTIF:
    df_PO_OTIF[['ON_TIME', 'IN_FULL', 'OTIF']] = pd.DataFrame([["","",""]], index=df_PO_OTIF.index)
    #df_PO_OTIF = df_PO_OTIF.reindex(columns = df_PO_OTIF.columns.tolist() + ['ON_TIME', 'IN_FULL', 'OTIF'])
        #add column via reindex > column is filled with value NaN
else:
    #filter the null values > in order to mark OTIF status
    Blank_filter = (df_PO_OTIF['OTIF'].isnull()) | (df_PO_OTIF['OTIF'].str.len() == 0) 
    df_PO_OTIF = df_PO_OTIF[Blank_filter]
                            
df_PO_OTIF

,PONbr,POLineNbr,QtyOrd,QtyRcvd,PromDate,ON_TIME,IN_FULL,OTIF
6,A000005177,1,2700.0,2700.0,2021-04-20,,,
7,A000005022,1,1000.0,1000.0,2021-04-06,,,
8,A000004594,1,500.0,500.0,2021-02-08,,,
9,A000004491,1,10000.0,10003.0,2021-01-15,,,
10,A000004429,1,2000.0,1988.3,2021-01-30,,,
...,...,...,...,...,...,...,...,...
4202,A000003984,1,1.0,1.0,2021-01-21,,,
4203,A000003984,2,2.0,2.0,2021-01-21,,,
4204,A000003984,3,2.0,2.0,2021-01-21,,,
4205,A000003984,4,1.0,1.0,2021-01-21,,,


## Bước 3. ON TIME - IN FULL

In [5]:
import numpy as np

df_PO_OTIF ['ON_TIME'] = np.where(df_PO_OTIF['QtyRcvd']>0, 'YES', 'NO')
#df_PO_OTIF [df_PO_OTIF['ON_TIME'] == 'NO']
df_PO_OTIF.head()

,PONbr,POLineNbr,QtyOrd,QtyRcvd,PromDate,ON_TIME,IN_FULL,OTIF
6,A000005177,1,2700.0,2700.0,2021-04-20,YES,,
7,A000005022,1,1000.0,1000.0,2021-04-06,YES,,
8,A000004594,1,500.0,500.0,2021-02-08,YES,,
9,A000004491,1,10000.0,10003.0,2021-01-15,YES,,
10,A000004429,1,2000.0,1988.3,2021-01-30,YES,,


In [6]:
### Quantity Tolerance

policy_tolerance = 0.05

cond_tolerance = (((df_PO_OTIF ['QtyRcvd'] - df_PO_OTIF ['QtyOrd'])/df_PO_OTIF ['QtyOrd']) <= policy_tolerance) &  \
    (((df_PO_OTIF ['QtyRcvd'] - df_PO_OTIF ['QtyOrd'])/df_PO_OTIF ['QtyOrd']) >= -policy_tolerance)

df_PO_OTIF ['IN_FULL'] = np.where(cond_tolerance, 'YES', 'NO')
#df_PO_OTIF [df_PO_OTIF['IN_FULL'] == 'NO']
df_PO_OTIF.head()

,PONbr,POLineNbr,QtyOrd,QtyRcvd,PromDate,ON_TIME,IN_FULL,OTIF
6,A000005177,1,2700.0,2700.0,2021-04-20,YES,YES,
7,A000005022,1,1000.0,1000.0,2021-04-06,YES,YES,
8,A000004594,1,500.0,500.0,2021-02-08,YES,YES,
9,A000004491,1,10000.0,10003.0,2021-01-15,YES,YES,
10,A000004429,1,2000.0,1988.3,2021-01-30,YES,YES,


In [7]:
### Xét OTIF dựa trên giá trị của 2 cột ON TIME và IN FULL

cond_otif = (df_PO_OTIF ['ON_TIME'] == 'YES') & (df_PO_OTIF ['IN_FULL'] == 'YES')
df_PO_OTIF ['OTIF'] = np.where(cond_otif, 'YES', 'NO')
#df_PO_OTIF [df_PO_OTIF['OTIF'] == 'NO']
df_PO_OTIF.head()

,PONbr,POLineNbr,QtyOrd,QtyRcvd,PromDate,ON_TIME,IN_FULL,OTIF
6,A000005177,1,2700.0,2700.0,2021-04-20,YES,YES,YES
7,A000005022,1,1000.0,1000.0,2021-04-06,YES,YES,YES
8,A000004594,1,500.0,500.0,2021-02-08,YES,YES,YES
9,A000004491,1,10000.0,10003.0,2021-01-15,YES,YES,YES
10,A000004429,1,2000.0,1988.3,2021-01-30,YES,YES,YES


#### Combined ON TIME - IN FULL (BYPASS SUB-COLUMN)

## Bước 4.  JOIN VỚI DỮ LIỆU GỐC

In [16]:
df_PO_OTIF_join = df_PO.merge(df_PO_OTIF, how = 'left')
df_PO_OTIF_join [['ON_TIME', 'IN_FULL', 'OTIF']] = df_PO_OTIF_join[['ON_TIME', 'IN_FULL', 'OTIF']].fillna("")
df_PO_OTIF_join.head()

,PONbr,POLineNbr,PR_NBR_REF,PR_LINE_REF,SiteID,InvtID,Trandesc,Super_Category,CuryID,UnitCost,...,VendorName,Status_Rcpt,Status_Voucher,TradeTerm,Status_PO,Crtd_user_PO,Crtd_Date_PO,ON_TIME,IN_FULL,OTIF
0,A000007101,1,FD-14090,7,02GIAVI,A20019,"Xe đua F - Đen, 66",Sản phẩm_Xe đạp,VND,120000.0,...,Cong Ty Co Phan Thuong Mai Dich Vu Ngoc An,Full Receipt,Fully Vouchered,Delivery At Seller,Completed,HADM,2021-08-15,,,
1,A000007101,2,FD-14090,10,02GIAVI,A20015,"Khung xe đua LL - Đỏ, 60",Phụ tùng_Xe đạp,VND,76500.0,...,Cong Ty Co Phan Thuong Mai Dich Vu Ngoc An,Full Receipt,Fully Vouchered,Delivery At Seller,Completed,HADM,2021-08-15,,,
2,A000007079,1,FD-13978,9,02FDNF,D10106,"Khung xe đường phố LL - Vàng, 66",Phụ tùng_Xe đạp,VND,74000.0,...,Cong ty Co Phan Quang Ninh,Full Receipt,Not Vouchered,Delivery At Seller,Completed,LAMDT,2021-08-14,,,
3,A000006708,1,FD-13763,2,02FDNF,D10090,"Xe leo núi-B Bạc, 38",Sản phẩm_Xe đạp,VND,69000.0,...,Cong Ty TNHH Hung Phuoc Loi,Partially Receipt,Not Vouchered,Delivery At Seller,Open Order,LAMDT,2021-07-22,,,
4,A000005968,1,FD-13531,1,02FDNF,D10109,"Xe đua G - Đen, 58",Sản phẩm_Xe đạp,VND,33000.0,...,Cong ty Co Phan Quang Ninh,Full Receipt,Fully Vouchered,Delivery At Seller,Completed,LAMDT,2021-06-16,,,


In [15]:
df_PO_OTIF_join.to_excel('OUTPUT_OTIF.xlsx')

## Bước 5. Tính OTIF theo POLineNbr

In [42]:
### Tính số OTIF Line theo Vendor

#Cách 1
#df_OTIF_by_vendor = df_PO_OTIF_join.groupby(['VendorName'])[['OTIF']].apply(lambda x: x[x['OTIF'] == 'YES' ]).count()
#df_OTIF_by_vendor = df_PO_OTIF_join.groupby(['VendorName']).apply(lambda x: x[x['OTIF'] == 'YES' ][['OTIF']].count())

#Cách 2
#import numpy as np
#df_OTIF_by_vendor = df_PO_OTIF_join.assign(OTIF_line = np.where(df_PO_OTIF_join['OTIF']=='YES',1,0))\
#.groupby('VendorName').agg({'OTIF_line':'sum'})

#Cách 3
df_OTIFLine_by_vendor=df_PO_OTIF_join[df_PO_OTIF_join['OTIF']=='YES'].groupby(['VendorName'])[['OTIF']].count()

df_OTIFLine_by_vendor

,OTIF
VendorName,
Cong Ty CP The Gioi Vo Xe,1
Cong Ty CP Vat Tu & Dich Vu Ky Thuat Thanh Pho Ho Chi Minh,1
Cong Ty Co Phan Azuma Viet Nam,3
Cong Ty Co Phan Cat Duong,2
Cong Ty Co Phan Cong Nghe An Toan Viet Nam,3
...,...
Cong ty TNHH Thuong Mai Taiyo,121
Cong ty co phan Long Thanh,1
Cua Hang Phu Tung O To 9 Nghia,2


In [49]:
### Tính số POLine theo Vendor (không tính những line chưa xét OTIF)

#Cách 1
#df_OTIF_by_vendor = df_PO_OTIF_join.groupby(['VendorName'])[['OTIF']].apply(lambda x: x[x['OTIF'].str.len() != 0 ]).count()
#df_OTIF_by_vendor = df_PO_OTIF_join.groupby(['VendorName']).apply(lambda x: x[x['OTIF'].str.len() != 0 ][['OTIF']].count())

#Cách 2
#import numpy as np
#df_OTIF_by_vendor = df_PO_OTIF_join.assign(OTIF_line = np.where(df_PO_OTIF_join['OTIF'].str.len() != 0,1,0))\
#.groupby('VendorName').agg({'OTIF_line':'sum'})

#Cách 3
df_POLine_by_vendor=df_PO_OTIF_join[df_PO_OTIF_join['OTIF'].str.len() != 0].groupby(['VendorName'])[['OTIF']].count()
#notnull không work

df_POLine_by_vendor

,OTIF
VendorName,
Cong Ty CP The Gioi Vo Xe,1
Cong Ty CP Vat Tu & Dich Vu Ky Thuat Thanh Pho Ho Chi Minh,1
Cong Ty Co Phan Armephaco,4
Cong Ty Co Phan Azuma Viet Nam,3
Cong Ty Co Phan Cat Duong,2
...,...
Cong ty co phan Long Thanh,1
Cua Hang Phu Tung O To 9 Nghia,2
Doanh Nghiep Tu Nhan Nhua Thuy Duong,4


In [100]:
### Tính chỉ số OTIF theo VendorName

df_OTIF_rate_by_Vendor = df_OTIFLine_by_vendor.div(df_POLine_by_vendor, fill_value = 0)
df_OTIF_rate_by_Vendor.rename(columns = {'OTIF':'OTIF_rate_by_POLineNbr'}, inplace = True)
pd.options.display.float_format = '{:.0%}'.format
df_OTIF_rate_by_Vendor
#df_OTIF_rate_by_Vendor.to_excel('OUTPUT_OTIF_test.xlsx')

,OTIF_rate_by_POLineNbr
VendorName,
Cong Ty CP The Gioi Vo Xe,100%
Cong Ty CP Vat Tu & Dich Vu Ky Thuat Thanh Pho Ho Chi Minh,100%
Cong Ty Co Phan Armephaco,0%
Cong Ty Co Phan Azuma Viet Nam,100%
Cong Ty Co Phan Cat Duong,100%
...,...
Cong ty co phan Long Thanh,100%
Cua Hang Phu Tung O To 9 Nghia,100%
Doanh Nghiep Tu Nhan Nhua Thuy Duong,100%


## Bước 5' (On-demand). Tính OTIF theo PONbr

In [90]:
### Tính số OTIF PONbr theo Vendor

#Lấy list PONbr mà trong đó tất cả các Line đều đạt OTIF
#df_OTIF_PO_by_vendor = df_PO_OTIF_join.groupby(['PONbr','VendorName']).filter(lambda x: x[x['OTIF'] == 'YES'].all())   > why not work?
df_OTIF_PO_by_vendor = df_PO_OTIF_join.groupby(['PONbr','VendorName']).filter(lambda x: x['OTIF'].isin(['YES']).all())

#df_OTIF_PO_by_vendor.to_excel('OUTPUT_OTIF_test.xlsx')

#Tính số PONbr by Vendor đạt OTIF
df_OTIF_PO_by_vendor_Quantity = df_OTIF_PO_by_vendor.groupby(['VendorName'])[['PONbr']].nunique()
df_OTIF_PO_by_vendor_Quantity

,PONbr
VendorName,
Cong Ty CP The Gioi Vo Xe,1
Cong Ty Co Phan Azuma Viet Nam,1
Cong Ty Co Phan Cat Duong,1
Cong Ty Co Phan Cong Nghe An Toan Viet Nam,2
Cong Ty Co Phan Dac Kim,18
...,...
Cong ty TNHH Thuong Mai Taiyo,11
Cong ty co phan Long Thanh,1
Cua Hang Phu Tung O To 9 Nghia,2


In [95]:
#Tính số PONbr by Vendor (không tính PONbr có line chưa xét OTIF)

#Lấy list PONbr mà trong đó tất cả các Line đều có xét OTIF
df_PONbr_by_vendor = df_PO_OTIF_join.groupby(['PONbr','VendorName']).filter(lambda x: x['OTIF'].isin(['YES', 'NO']).all())
#df_PONbr_by_vendor.to_excel('OUTPUT_OTIF_test.xlsx')

#Tính số lượng PONbr by Vendor đã xét đủ OTIF
df_PONbr_by_vendor_Quantity = df_PONbr_by_vendor.groupby(['VendorName'])[['PONbr']].nunique()
df_PONbr_by_vendor_Quantity

,PONbr
VendorName,
Cong Ty CP The Gioi Vo Xe,1
Cong Ty Co Phan Armephaco,3
Cong Ty Co Phan Azuma Viet Nam,1
Cong Ty Co Phan Cat Duong,1
Cong Ty Co Phan Cong Nghe An Toan Viet Nam,3
...,...
Cong ty co phan Long Thanh,1
Cua Hang Phu Tung O To 9 Nghia,2
Doanh Nghiep Tu Nhan Nhua Thuy Duong,4


In [99]:
### Tính chỉ số OTIF theo VendorName, PONbr

df_OTIF_rate_by_Vendor_PONbr = df_OTIF_PO_by_vendor_Quantity.div(df_PONbr_by_vendor_Quantity, fill_value = 0)
df_OTIF_rate_by_Vendor_PONbr.rename(columns = {'PONbr':'OTIF_rate_by_PONbr'}, inplace = True)
pd.options.display.float_format = '{:.0%}'.format
df_OTIF_rate_by_Vendor_PONbr

,OTIF_rate_by_PONbr
VendorName,
Cong Ty CP The Gioi Vo Xe,100%
Cong Ty Co Phan Armephaco,0%
Cong Ty Co Phan Azuma Viet Nam,100%
Cong Ty Co Phan Cat Duong,100%
Cong Ty Co Phan Cong Nghe An Toan Viet Nam,67%
...,...
Cong ty co phan Long Thanh,100%
Cua Hang Phu Tung O To 9 Nghia,100%
Doanh Nghiep Tu Nhan Nhua Thuy Duong,100%
